In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Fixed car specifications for new makes/models
fixed_values = {
    1: {"Make": "Maruti Jimny EV", "mgr": "Ananya", "Top Speed": 140, "Total Power": 200, "Total Torque": 300, "Drive": "AWD", 
        "BC": 40, "Length": 3985, "Width": 1645, "Height": 1720, "Wheelbase": 2590, "Gross Vehicle Weight (GVWR)": 1450, 
        "Max. Payload": 400, "Cargo Volume": 220, "Seats": 4},
    2: {"Make": "Nissan ARIYA", "mgr": "Ramesh", "Top Speed": 160, "Total Power": 290, "Total Torque": 450, "Drive": "AWD", 
        "BC": 63, "Length": 4595, "Width": 1850, "Height": 1660, "Wheelbase": 2775, "Gross Vehicle Weight (GVWR)": 1980, 
        "Max. Payload": 480, "Cargo Volume": 466, "Seats": 5},
    3: {"Make": "Honda Clarity EV", "mgr": "Sneha", "Top Speed": 180, "Total Power": 240, "Total Torque": 400, "Drive": "Rear", 
        "BC": 80, "Length": 4895, "Width": 1870, "Height": 1490, "Wheelbase": 2760, "Gross Vehicle Weight (GVWR)": 2100, 
        "Max. Payload": 450, "Cargo Volume": 470, "Seats": 5},
    4: {"Make": "Hyundai Ioniq 5", "mgr": "Vikram", "Top Speed": 185, "Total Power": 320, "Total Torque": 605, "Drive": "Rear", 
        "BC": 77.4, "Length": 4635, "Width": 1890, "Height": 1605, "Wheelbase": 3000, "Gross Vehicle Weight (GVWR)": 2200, 
        "Max. Payload": 500, "Cargo Volume": 531, "Seats": 5},
    5: {"Make": "Citroen eC3", "mgr": "Maya", "Top Speed": 110, "Total Power": 90, "Total Torque": 200, "Drive": "Front", 
        "BC": 29.2, "Length": 3981, "Width": 1733, "Height": 1585, "Wheelbase": 2540, "Gross Vehicle Weight (GVWR)": 1360, 
        "Max. Payload": 350, "Cargo Volume": 315, "Seats": 5}
}

# Fixed maintenance costs for each make
fixed_maintenance_costs = {
    "Maruti Jimny EV": 75,
    "Nissan ARIYA": 100,
    "Honda Clarity EV": 120,
    "Hyundai Ioniq 5": 130,
    "Citroen eC3": 80
}

# Driver and manager pool
drivers = ["Alice", "Bob", "Charlie", "Diana", "Ethan", "Fiona", "George", "Hannah", "Ivan", "Julia"]

# Generate relationships for dynamic values
def calculate_acceleration(top_speed, driver_skill):
    base_acceleration = 10 / (top_speed / 100)  # Basic relation
    skill_factor = 1 - (driver_skill / 10)  # Better drivers lower acceleration time
    return round(base_acceleration * skill_factor, 2)

def calculate_er(bc, terrain_factor):
    base_er = bc * 5  # Assume 1 kWh gives 5 miles
    terrain_penalty = base_er * (terrain_factor / 100)  # Reduce range based on terrain
    return round(base_er - terrain_penalty, 1)

def calculate_charging_cost(bc):
    """Calculate charging cost based on battery capacity"""
    return round(bc * 0.15, 2)  # Assume cost per kWh is 0.15 USD

# Date range for the dataset (1st Jan 2023 to 31st Dec 2023)
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)

# Generate dataset
n = 1000
data = []

for i in range(n):
    car_id = random.randint(1, 5)
    car = fixed_values[car_id]
    
    driver = random.choice(drivers)
    driver_skill = random.randint(5, 10)
    terrain_factor = random.uniform(5, 15)
    
    make = car["Make"]
    mgr = car["mgr"]
    top_speed = car["Top Speed"]
    bc = car["BC"]
    
    acceleration = calculate_acceleration(top_speed, driver_skill)
    er = calculate_er(bc, terrain_factor)
    charging_cost = calculate_charging_cost(bc)
    
    # Use the fixed maintenance cost based on the car make
    maintenance_cost = fixed_maintenance_costs.get(make, 100)
    
    total_power = car["Total Power"]
    total_torque = car["Total Torque"]
    drive = car["Drive"]
    length = car["Length"]
    width = car["Width"]
    height = car["Height"]
    wheelbase = car["Wheelbase"]
    gvwr = car["Gross Vehicle Weight (GVWR)"]
    max_payload = car["Max. Payload"]
    cargo_volume = car["Cargo Volume"]
    seats = car["Seats"]
    
    # Random date within the specified range
    random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    formatted_date = random_date.strftime("%d-%m-%Y")
    
    # Adding data to the list
    data.append([i+1, make, driver, mgr, acceleration, top_speed, er, total_power, total_torque, drive, 
                 bc, length, width, height, wheelbase, gvwr, max_payload, cargo_volume, seats, 
                 charging_cost, maintenance_cost, formatted_date])

# Create DataFrame
columns = ["id", "Make", "Driver", "Manager", "Acceleration 0 - 100 km/h", "Top Speed", "ER", 
           "Total Power", "Total Torque", "Drive", "BC", "Length", "Width", "Height", 
           "Wheelbase", "Gross Vehicle Weight (GVWR)", "Max. Payload", "Cargo Volume", "Seats", 
           "Charging Cost", "Maintenance Cost", "Date"]
df = pd.DataFrame(data, columns=columns)

# Add the new 'Range' column
df['Range'] = (df['ER'] * random.uniform(0.8, 1.2)).round(2)

# Save to CSV
df.to_csv("synthetic_ev_dataset_with_updated_range.csv", index=False)

# Calculate monthly maintenance and charging costs
df['Month'] = pd.to_datetime(df['Date'], format="%d-%m-%Y").dt.month
monthly_costs = df.groupby(['Make', 'Month'])[['Charging Cost', 'Maintenance Cost']].sum().reset_index()

# Get the most recent record for each make
df['Date'] = pd.to_datetime(df['Date'], format="%d-%m-%Y")
last_record = df.loc[df.groupby('Make')['Date'].idxmax()]

# Print results
print("Synthetic dataset generated and saved as 'synthetic_ev_dataset_with_updated_range.csv'.")
print("\nMonthly Charging and Maintenance Costs:\n", monthly_costs)
print("\nLast record for each make:\n", last_record)


Synthetic dataset generated and saved as 'synthetic_ev_dataset_with_updated_range.csv'.

Monthly Charging and Maintenance Costs:
                 Make  Month  Charging Cost  Maintenance Cost
0        Citroen eC3      1          61.32              1120
1        Citroen eC3      2          65.70              1200
2        Citroen eC3      3          83.22              1520
3        Citroen eC3      4          70.08              1280
4        Citroen eC3      5          91.98              1680
5        Citroen eC3      6          74.46              1360
6        Citroen eC3      7          87.60              1600
7        Citroen eC3      8         100.74              1840
8        Citroen eC3      9          70.08              1280
9        Citroen eC3     10          56.94              1040
10       Citroen eC3     11          61.32              1120
11       Citroen eC3     12          70.08              1280
12  Honda Clarity EV      1         144.00              1440
13  Honda Clarit

In [2]:
df


,id,Make,Driver,Manager,Acceleration 0 - 100 km/h,Top Speed,ER,Total Power,Total Torque,Drive,...,Wheelbase,Gross Vehicle Weight (GVWR),Max. Payload,Cargo Volume,Seats,Charging Cost,Maintenance Cost,Date,Range,Month
0,1,Honda Clarity EV,Alice,Sneha,0.00,180,356.8,240,400,Rear,...,2760,2100,450,470,5,12.00,120,2023-08-27,351.90,8
1,2,Maruti Jimny EV,Julia,Ananya,2.14,140,173.9,200,300,AWD,...,2590,1450,400,220,4,6.00,75,2023-11-18,171.51,11
2,3,Citroen eC3,Charlie,Maya,1.82,110,127.5,90,200,Front,...,2540,1360,350,315,5,4.38,80,2023-02-23,125.75,2
3,4,Maruti Jimny EV,Hannah,Ananya,3.57,140,175.8,200,300,AWD,...,2590,1450,400,220,4,6.00,75,2023-12-17,173.38,12
4,5,Maruti Jimny EV,Fiona,Ananya,3.57,140,186.5,200,300,AWD,...,2590,1450,400,220,4,6.00,75,2023-04-27,183.94,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Citroen eC3,Diana,Maya,0.00,110,124.6,90,200,Front,...,2540,1360,350,315,5,4.38,80,2023-09-29,122.89,9
996,997,Honda Clarity EV,Fiona,Sneha,0.00,180,374.9,240,400,Rear,...,2760,2100,450,470,5,12.00,120,2023-12-30,369.75,12
997,998,Nissan ARIYA,Fiona,Ramesh,3.12,160,270.0,290,450,AWD,...,2775,1980,480,466,5,9.45,100,2023-02-13,266.29,2
998,999,Citroen eC3,Hannah,Maya,3.64,110,137.0,90,200,Front,...,2540,1360,350,315,5,4.38,80,2023-12-31,135.12,12


In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Fixed car specifications for new makes/models
fixed_values = {
    1: {"Make": "Maruti Jimny EV", "mgr": "Ananya", "Top Speed": 140, "Total Power": 200, "Total Torque": 300, "Drive": "AWD", 
        "BC": 40, "Length": 3985, "Width": 1645, "Height": 1720, "Wheelbase": 2590, "Gross Vehicle Weight (GVWR)": 1450, 
        "Max. Payload": 400, "Cargo Volume": 220, "Seats": 4},
    2: {"Make": "Nissan ARIYA", "mgr": "Ramesh", "Top Speed": 160, "Total Power": 290, "Total Torque": 450, "Drive": "AWD", 
        "BC": 63, "Length": 4595, "Width": 1850, "Height": 1660, "Wheelbase": 2775, "Gross Vehicle Weight (GVWR)": 1980, 
        "Max. Payload": 480, "Cargo Volume": 466, "Seats": 5},
    3: {"Make": "Honda Clarity EV", "mgr": "Sneha", "Top Speed": 180, "Total Power": 240, "Total Torque": 400, "Drive": "Rear", 
        "BC": 80, "Length": 4895, "Width": 1870, "Height": 1490, "Wheelbase": 2760, "Gross Vehicle Weight (GVWR)": 2100, 
        "Max. Payload": 450, "Cargo Volume": 470, "Seats": 5},
    4: {"Make": "Hyundai Ioniq 5", "mgr": "Vikram", "Top Speed": 185, "Total Power": 320, "Total Torque": 605, "Drive": "Rear", 
        "BC": 77.4, "Length": 4635, "Width": 1890, "Height": 1605, "Wheelbase": 3000, "Gross Vehicle Weight (GVWR)": 2200, 
        "Max. Payload": 500, "Cargo Volume": 531, "Seats": 5},
    5: {"Make": "Citroen eC3", "mgr": "Maya", "Top Speed": 110, "Total Power": 90, "Total Torque": 200, "Drive": "Front", 
        "BC": 29.2, "Length": 3981, "Width": 1733, "Height": 1585, "Wheelbase": 2540, "Gross Vehicle Weight (GVWR)": 1360, 
        "Max. Payload": 350, "Cargo Volume": 315, "Seats": 5}
}

# Fixed maintenance costs for each make
fixed_maintenance_costs = {
    "Maruti Jimny EV": 75,
    "Nissan ARIYA": 100,
    "Honda Clarity EV": 120,
    "Hyundai Ioniq 5": 130,
    "Citroen eC3": 80
}

# Driver and manager pool
drivers = ["Alice", "Bob", "Charlie", "Diana", "Ethan", "Fiona", "George", "Hannah", "Ivan", "Julia"]

# Helper functions
def calculate_acceleration(top_speed, driver_skill):
    base_acceleration = 10 / (top_speed / 100)
    skill_factor = 1 - (driver_skill / 10)
    return round(base_acceleration * skill_factor, 2)

def calculate_er(bc, terrain_factor):
    base_er = bc * 5
    terrain_penalty = base_er * (terrain_factor / 100)
    return round(base_er - terrain_penalty, 1)

def calculate_charging_cost(bc):
    return round(bc * 0.15, 2)

# Generate dataset
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)
n = 1000
data = []

for i in range(n):
    car_id = random.randint(1, 5)
    car = fixed_values[car_id]
    
    driver = random.choice(drivers)
    driver_skill = random.randint(5, 10)
    terrain_factor = random.uniform(5, 15)
    
    make = car["Make"]
    mgr = car["mgr"]
    top_speed = car["Top Speed"]
    bc = car["BC"]
    
    acceleration = calculate_acceleration(top_speed, driver_skill)
    er = calculate_er(bc, terrain_factor)
    charging_cost = calculate_charging_cost(bc)
    maintenance_cost = fixed_maintenance_costs.get(make, 100)
    
    random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    formatted_date = random_date.strftime("%d-%m-%Y")
    
    data.append([i+1, make, driver, mgr, acceleration, top_speed, er, car["Total Power"], car["Total Torque"], car["Drive"], 
                 bc, car["Length"], car["Width"], car["Height"], car["Wheelbase"], car["Gross Vehicle Weight (GVWR)"], 
                 car["Max. Payload"], car["Cargo Volume"], car["Seats"], charging_cost, maintenance_cost, formatted_date])

columns = ["id", "Make", "Driver", "Manager", "Acceleration 0 - 100 km/h", "Top Speed", "ER", 
           "Total Power", "Total Torque", "Drive", "BC", "Length", "Width", "Height", 
           "Wheelbase", "Gross Vehicle Weight (GVWR)", "Max. Payload", "Cargo Volume", "Seats", 
           "Charging Cost", "Maintenance Cost", "Date"]

df = pd.DataFrame(data, columns=columns)
df['Range'] = (df['ER'] * random.uniform(0.8, 1.2)).round(2)
df['Date'] = pd.to_datetime(df['Date'], format="%d-%m-%Y")
df['Month'] = df['Date'].dt.month

df.to_csv("synthetic_ev_dataset_with_updated_range.csv", index=False)


In [2]:
df


,id,Make,Driver,Manager,Acceleration 0 - 100 km/h,Top Speed,ER,Total Power,Total Torque,Drive,...,Wheelbase,Gross Vehicle Weight (GVWR),Max. Payload,Cargo Volume,Seats,Charging Cost,Maintenance Cost,Date,Range,Month
0,1,Nissan ARIYA,Alice,Ramesh,3.12,160,295.3,290,450,AWD,...,2775,1980,480,466,5,9.45,100,2023-10-31,328.15,10
1,2,Nissan ARIYA,Alice,Ramesh,1.88,160,270.8,290,450,AWD,...,2775,1980,480,466,5,9.45,100,2023-02-09,300.93,2
2,3,Nissan ARIYA,Charlie,Ramesh,0.00,160,269.0,290,450,AWD,...,2775,1980,480,466,5,9.45,100,2023-05-17,298.93,5
3,4,Honda Clarity EV,Ethan,Sneha,1.67,180,373.4,240,400,Rear,...,2760,2100,450,470,5,12.00,120,2023-03-28,414.94,3
4,5,Hyundai Ioniq 5,Charlie,Vikram,2.16,185,334.1,320,605,Rear,...,3000,2200,500,531,5,11.61,130,2023-04-02,371.27,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Citroen eC3,Ivan,Maya,1.82,110,132.5,90,200,Front,...,2540,1360,350,315,5,4.38,80,2023-10-23,147.24,10
996,997,Honda Clarity EV,Ethan,Sneha,0.00,180,349.0,240,400,Rear,...,2760,2100,450,470,5,12.00,120,2023-01-04,387.83,1
997,998,Honda Clarity EV,George,Sneha,1.67,180,340.9,240,400,Rear,...,2760,2100,450,470,5,12.00,120,2023-11-14,378.82,11
998,999,Maruti Jimny EV,Ivan,Ananya,0.00,140,174.2,200,300,AWD,...,2590,1450,400,220,4,6.00,75,2023-07-03,193.58,7
